In [1]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import equistore
from equistore import Labels, TensorBlock, TensorMap

# from rascaline.utils import clebsch_gordan
import clebsch_gordan
import spherical

In [2]:
frames = ase.io.read("combined_magres_spherical.xyz", ":")

# Define hyperparameters for generating the rascaline SphericalExpansion
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

# Define target lambda channels
lambdas = np.array([0, 1, 2])

# Equivariance Test

In [3]:
# Pick a test frame
frame = frames[0]

# Randomly rigidly rotate the frame
frame_rot, (a, b, c) = spherical.rotate_ase_frame(frame)
print("Random rotation angles (rad):", a, b, c)

# Generate lambda-SOAP for both frames
lsoap = clebsch_gordan.n_body_iteration_single_center(
    [frame],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

lsoap_rot = clebsch_gordan.n_body_iteration_single_center(
    [frame_rot],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

# Build a Wigner-D Matrix from the random rotation angles
wig = spherical.WignerDReal(rascal_hypers["max_angular"], a, b, c)

# Rotate the lambda-SOAP descriptor of the unrotated frame
lsoap_unrot_rot = wig.rotate_tensormap(lsoap)

Random rotation angles (rad): 2.772363661827398 1.8039589759258468 1.2427261014900488


In [6]:
assert equistore.equal_metadata(lsoap_unrot_rot, lsoap_rot)
assert equistore.allclose(lsoap_unrot_rot, lsoap_rot)

# Normalization Tests

In [ ]:
lsoap_new0 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)
lsoap_new0

In [ ]:
lsoap_old0 = spherical.lambda_soap_vector(
    frames,
    rascal_hypers,
)
# Drop all odd parity keys/blocks
keys_to_drop = Labels(
    names=lsoap_old0.keys.names,
    values=lsoap_old0.keys.values[[v not in lambdas for v in lsoap_old0.keys.column("spherical_harmonics_l")]],
)
lsoap_old0 = equistore.drop_blocks(lsoap_old0, keys=keys_to_drop)
lsoap_old0

# Test system 1

In [ ]:
%%timeit

use_sparse = False

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_dense

# timeit comes out at about 22 seconds for nu_target = 3

In [ ]:
%%timeit

use_sparse = True

tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_sparse

# timeit comes out at about 13 seconds for nu_target = 3

In [ ]:
tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=True,
)

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=False,
)

assert equistore.allclose(tensor_dense, tensor_sparse)

# Test system 2

In [ ]:
frames = [ase.io.read("frame.xyz")]

lambdas = np.array([0, 2])

rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

n_body = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)
n_body

In [ ]:
# selected_samples = None
# species_neighbors = [1, 8, 6]

# calculator = rascaline.SphericalExpansion(**rascal_hypers)
# nu1 = calculator.compute(frames, selected_samples=selected_samples)

# # Move the "species_neighbor" key to the properties. If species_neighbors is
# # passed as a list of int, sparsity can be created in the properties for
# # these species.
# if species_neighbors is None:
#     keys_to_move = "species_neighbor"
# else:
#     keys_to_move = Labels(
#         names=["species_neighbor"],
#         values=np.array(species_neighbors).reshape(-1, 1),
#     )
# nu1 = nu1.keys_to_properties(keys_to_move=keys_to_move)
# nu1 = clebsch_gordan._add_nu_sigma_to_key_names(nu1)